In [1]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [2]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

In [3]:
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [4]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 32
opt.gpu_num = 1
opt.model = "x3d"
opt.model_depth = 50
opt.clip_duarion = 1
opt.version = '0724' + '_' + opt.model + '_depth' + str(opt.model_depth)


In [5]:
from utils.utils import get_ckpt_path

model = WalkVideoClassificationLightningModule(opt)

# get last ckpt path
ckpt_path = get_ckpt_path(opt)


model = model.load_from_checkpoint(ckpt_path)

model.eval()

clear_output()
print(ckpt_path)


/workspace/Walk_Video_PyTorch/logs/x3d/0724_x3d_depth50/checkpoints/epoch=99-step=9988.ckpt


In [6]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)

trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=1,
    deterministic=True,
)

trainer.test(dataloaders=module, model=model)


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:460: UserWarning: The flag `devices=1` will be ignored, instead the device specific number 1 will be used
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 19. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 27. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection.

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.20969441533088684    │
│      test_loss_epoch      │     1.087226390838623     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 1.087226390838623, 'test_acc_epoch': 0.20969441533088684}]

In [7]:
# trainer.validate(model=model, dataloaders=module)

In [8]:
input_data = next(iter(test_data))


In [9]:
input_data['video'].shape

input_data["video_name"]

input_data['video'][0][0][0][0]

tensor([-0.4139,  0.8608,  0.8478,  0.8235,  0.8887,  0.9493,  0.8629,  0.9736,
         0.9662,  1.2505,  1.6023,  1.2303, -0.2963, -0.2416, -0.1844, -0.1351,
        -0.1176, -0.1176, -0.0566,  0.0990,  1.3954,  1.6166,  2.0716,  2.3634,
         2.4096,  2.4096,  2.4096,  1.5312, -0.0915, -0.7800, -1.1871, -0.4486,
         0.6138,  2.4096,  2.3334,  1.6189,  1.5599,  2.2723,  0.3523,  1.9796,
         1.3100,  1.2151,  1.5797,  2.4287,  2.4202,  2.4096,  2.4096,  2.4096,
         2.4096,  2.4096,  2.4019,  0.8952,  0.3395,  0.3018,  0.2789,  0.2832,
         0.2484,  0.2484,  0.1786,  0.1786,  0.1786,  0.1786,  0.1612,  0.1612,
         0.1612,  0.1612,  0.1612,  0.2179,  0.1786,  0.1786,  0.1961,  0.2033,
         0.0802,  0.0103,  0.1828,  0.2783,  0.8648,  0.8685,  0.1737,  0.1379,
         0.1400,  0.1743,  0.1961,  0.1307,  0.2660,  0.0287,  0.0246,  0.0510,
         0.0266,  0.0438,  0.1120,  0.1786,  0.1786,  0.1786,  0.2309,  0.2431,
         0.2702,  0.2832,  0.2832,  0.28

In [10]:
input_data['label']

tensor([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 0, 0, 0, 0, 0])

In [11]:
preds = model(input_data['video'])

In [12]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [13]:
pred_classes

tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1]])

In [14]:
from torch import softmax


pred_chagne = softmax(preds, dim=-1)
pred_chagne

tensor([[0.6135, 0.3865],
        [0.6135, 0.3865],
        [0.6135, 0.3865],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3867, 0.6133],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.6135, 0.3865],
        [0.6134, 0.3866],
        [0.6135, 0.3865],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3866, 0.6134],
        [0.3876, 0.6124],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.6135, 0.3865],
        [0.6135, 0.3865],
        [0.6135, 0.3865],
        [0.6135, 0.3865],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135],
        [0.3865, 0.6135]], grad_fn=<SoftmaxBackward0>)

In [15]:
from utils.metrics import get_Accuracy

accuracy = get_Accuracy()

accuracy(pred_chagne, input_data['label'])

tensor(0.0312)

In [16]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [17]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [18]:
real_calss

['asd_not',
 'asd_not',
 'asd_not',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd_not',
 'asd',
 'asd',
 'asd',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd_not',
 'asd',
 'asd',
 'asd',
 'asd',
 'asd']

In [19]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

Predicted labels: asd, asd, asd, asd_not, asd_not, asd_not, asd_not, asd_not, asd_not, asd_not, asd_not, asd, asd, asd, asd_not, asd_not, asd_not, asd_not, asd_not, asd_not, asd_not, asd_not, asd_not, asd, asd, asd, asd, asd_not, asd_not, asd_not, asd_not, asd_not
real label: asd_not,asd_not,asd_not,asd,asd,asd,asd,asd_not,asd,asd,asd,asd_not,asd_not,asd_not,asd,asd,asd,asd,asd,asd,asd,asd,asd,asd_not,asd_not,asd_not,asd_not,asd,asd,asd,asd,asd


In [20]:
pred_class_names == real_calss

False

In [21]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result

['false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'true',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false']